In [1]:
import os
import argparse
import logging
logging.basicConfig(level=logging.DEBUG)
from common import find_mxnet, data, fit
from common.util import download_file
from ilab_iterator import ilab_iterator, Multi_ilab_iterator, Single_ilab_iterator, random_task_iterator
import mxnet as mx
import numpy

# class AccuracyIlab(mx.metric.EvalMetric):
#     """Calculate accuracies of multi label"""

#     def __init__(self):
#         super(Multi_Accuracy, self).__init__('multi-accuracy', num)

#     def update(self, labels, preds):
#         mx.metric.check_label_shapes(labels, preds)

# #         if self.num != None:
# #             assert len(labels) == self.num

#         for i in range(len(labels)):
#             pred_label = mx.nd.argmax_channel(preds[i]).asnumpy().astype('int32')
#             label = labels[i].asnumpy().astype('int32')

#             mx.metric.check_label_shapes(label, pred_label)

#             if i == None:
#                 self.sum_metric += (pred_label.flat == label.flat).sum()
#                 self.num_inst += len(pred_label.flat)
#             else:
#                 self.sum_metric[i] += (pred_label.flat == label.flat).sum()
#                 self.num_inst[i] += len(pred_label.flat)
            
class CrossEntropyIlab(mx.metric.EvalMetric):
    """Calculate Cross Entropy loss"""
    def __init__(self,task=0):
        super(CrossEntropyIlab, self).__init__('cross-entropy', task)
        self.task = task

    def update(self, labels, preds):
        mx.metric.check_label_shapes(labels, preds)

        for label, pred in zip(labels, preds):
            label = label[self.task].asnumpy().astype('int32')
            #label = label.asnumpy()
            pred = pred.asnumpy()

            label = label.ravel()
            assert label.shape[0] == pred.shape[0]
            prob = pred[numpy.arange(label.shape[0]), numpy.int64(label)]
            self.sum_metric += (-numpy.log(prob)).sum()
            self.num_inst += label.shape[0]
            f.open('trainig_stats.txt')
            f.write(self.sum_metric)
            f.close()

class Multi_Accuracy(mx.metric.EvalMetric):
    """Calculate accuracies of multi label"""

    def __init__(self, num=None):
        super(Multi_Accuracy, self).__init__('multi-accuracy', num)

    def update(self, labels, preds):
        #print labels 
        #print preds
        mx.metric.check_label_shapes(labels, preds)

        if self.num != None:
            assert len(labels) == self.num

        for i in range(len(labels)):
            pred_label = mx.nd.argmax_channel(preds[i]).asnumpy().astype('int32')
            label = labels[i].asnumpy().astype('int32')

            mx.metric.check_label_shapes(label, pred_label)

            if i == None:
                self.sum_metric += (pred_label.flat == label.flat).sum()
                self.num_inst += len(pred_label.flat)
            else:
                self.sum_metric[i] += (pred_label.flat == label.flat).sum()
                self.num_inst[i] += len(pred_label.flat)

class Multi_Entropy(mx.metric.EvalMetric):
    """Calculate accuracies of multi label"""

    def __init__(self, num=None):
        super(Multi_Entropy, self).__init__('multi-entropy', num)

    def update(self, labels, preds):
        #mx.metric.check_label_shapes(labels, preds)

        if self.num != None:
            print labels
            print len(labels)
            assert len(labels) == self.num

        for i in range(len(labels)):
                #pred_label = mx.nd.argmax_channel(preds[i]).asnumpy()
                #label = label.asnumpy()
                pred = preds[i].asnumpy()
                #pred = pred(pred_label)
                #prb = pred.ravel()
                label = labels[i].asnumpy().ravel()
                assert label.shape[0] == pred.shape[0]
                print label.shape, pred.shape
                prob = pred[numpy.arange(label.shape[0]), numpy.int64(label)]
                self.sum_metric[i] += (-numpy.log(prob)).sum()
                self.num_inst[i] += label.shape[0]         

class Cross_Entropy(mx.metric.EvalMetric):
    """Calculate accuracies of multi label"""

    def __init__(self):
        super(Cross_Entropy, self).__init__('single-entropy')
        self.sum_metric=[]
    def update(self, labels, preds):
        mx.metric.check_label_shapes(labels, preds)

        for j in range(len(labels)):
            label = labels[j].asnumpy()
            pred = preds[j].asnumpy()
            for i in range(label.shape[0]):
                prob = pred[i,numpy.int64(label[i])]
                if len(labels) == 1:
                    self.sum_metric += (-numpy.log(prob)).sum()
                else:
                    self.sum_metric[j] += (-numpy.log(prob)).sum()
            #print self.sum_metric
#                 #print len(labels)
#                 #pred_label = mx.nd.argmax_channel(preds[i]).asnumpy()
#                 #label = label.asnumpy()
#                 pred = preds[i].asnumpy()
#                 #pred = pred(pred_label)
#                 #prb = pred.ravel()
                
#                 assert label.shape[0] == pred.shape[0]
#                 print numpy.arange(label.shape[0]), label.shape, pred.shape
#                 prob = pred[:,numpy.int64(label)]
#                 print prob, prob.shape
#                 self.sum_metric += (-numpy.log(prob)).sum()
#                 self.num_inst += label.shape[0]  
                #print self.sum_metric
# if __name__ == '__main__':
# download data

# data setup
# parse args



In [2]:
image_shape = '3,224,224'
# load network
from importlib import import_module
#num_classes = [[10,0,0],[0,11,0],[0,0,8]]
num_classes=[10,11,8]

net = import_module('symbols.resnet_md')
batch_size = 64

train, val_raw = ilab_iterator(batch_size=batch_size)
train=random_task_iterator(train,subset=[0,2,3])#,num_cl=num_classes)
val=random_task_iterator(val_raw,subset=[0,2,3])#,num_cl=num_classes)
multi_val = Multi_ilab_iterator(val_raw,subset=[0,2,3])


def sym_gen(bucket_key):
    num_classes=[10,11,8]
    #num_classes=[10,10,10]
    active = [[1,0,0],[0,1,0],[0,0,1]]
    #num_classes = [[10,0],[0,8]]
    if bucket_key == 3:
        return net.get_vmt_symbol(num_classes, 50, image_shape, conv_workspace=256)
    else:
        return net.get_mt_symbol(num_classes,active[bucket_key], 50, image_shape, conv_workspace=256)

def shared_sym_gen():
    num_classes=[10,11,8]
    #num_classes=[10,118]
    #active = [[1,0,0],[0,1,0],[0,0,1]]
    #num_classes = [[10,0,0],[0,11,0],[0,0,8]]
    return net.get_vmt_symbol(num_classes, 50, image_shape, conv_workspace=256)
#num_classes = [10,91,11,8]
num_classes=[10,11,8]
#num_classes=[10]
#sym, label_names = net.get_vmt_symbol(num_classes, 50, image_shape, conv_workspace=256)
#mod = mx.mod.Module(sym, label_names=(label_names), context=[mx.gpu(0)])
mod = mx.mod.BucketingModule(sym_gen,default_bucket_key=3, context=[mx.gpu(0)])
#sym_gen, default_bucket_key=None
#mod = mx.mod.Module(sym, context=[mx.gpu(0)])



#val=random_task_iterator(mx.io.DataIter)
#train = Multi_ilab_iterator(train,subset=[0,2,3])
#val = Multi_ilab_iterator(val,subset=[0,2,3])
# train = Single_ilab_iterator(train,labid=1)
# val = Single_ilab_iterator(val,labid=1)

model_prefix = '/home/ubuntu/results/ilab10_rand'
#model_prefix = '/home/ubuntu/results/ilab_cat50'
checkpoint = mx.callback.do_checkpoint(model_prefix)




#lr_schedule it isimilar to the CIFAR100 schedule but half length of the steps
schedule = [20000,30000,40000]


In [3]:
#mon = mx.monitor.Monitor(1,pattern='softmax2_output')
#safety check to see whether all outputs are there
#sym.list_outputs()

Actually train the nets

In [4]:
mod.fit(train,
        eval_data=val,
        eval_metric=Cross_Entropy(),
        batch_end_callback = mx.callback.log_train_metric(100),
        epoch_end_callback=checkpoint,
        allow_missing=False,
        optimizer_params={'learning_rate':0.05, 'momentum': 0.9,'wd':0.0004, 'lr_scheduler': mx.lr_scheduler.MultiFactorScheduler(step=schedule,factor=0.1) },
        num_epoch=20)
# train loading form nepoch
# mod.fit(train, eval_data=val, epoch_end_callback=checkpoint, optimizer_params={'learning_rate':0.1, 'momentum': 0.9,'wd':0.0004}, num_epoch=300, arg_params=arg_params, aux_params=aux_params,
#         begin_epoch=n_epoch_load)


/home/ubuntu/mxnet/python/mxnet/module/base_module.py:64: UserWarning: Data provided by label_shapes don't match names specified by label_names ([DataDesc[softmax1_label,(64L,),<type 'numpy.float32'>,NCHW], DataDesc[softmax2_label,(64L,),<type 'numpy.float32'>,NCHW]] vs. ['softmax1_label', 'softmax2_label', 'softmax3_label'])
  warnings.warn(msg)


INFO:root:Iter[0] Batch[0] Train-single-entropy=nan


['[0] Eval metric = 2.40335542336 for bucket number 1']
['[1] Eval metric = 2.0776568912 for bucket number 2']
['[2] Eval metric = 2.32488366216 for bucket number 2']
['[3] Eval metric = 2.316926267 for bucket number 0']
['[4] Eval metric = 2.31554019265 for bucket number 2']
['[5] Eval metric = 2.41737364978 for bucket number 1']
['[6] Eval metric = 2.06407771434 for bucket number 0']
['[7] Eval metric = 2.3517560414 for bucket number 2']
['[8] Eval metric = 2.30957365129 for bucket number 0']
['[9] Eval metric = 3.19358519092 for bucket number 0']
['[10] Eval metric = 2.58327151602 for bucket number 1']
['[11] Eval metric = 2.82502782531 for bucket number 0']
['[12] Eval metric = 2.36306648143 for bucket number 0']
['[13] Eval metric = 2.83019426931 for bucket number 0']
['[14] Eval metric = 2.27912003919 for bucket number 2']
['[15] Eval metric = 2.50541706569 for bucket number 1']
['[16] Eval metric = 2.09087054944 for bucket number 2']
['[17] Eval metric = 2.07573045045 for bucket

KeyboardInterrupt: 

Get evaluation of multi accuracy and entropy on validation set at a specified epoch

In [5]:
#model_prefix = '/home/ubuntu/results/ilab10_rand_multi'
#model_prefix = '/home/ubuntu/results/ilab_cat50'
#checkpoint = mx.callback.do_checkpoint(model_prefix)
#sym, arg_params, aux_params = \
#       mx.model.load_checkpoint(model_prefix, 5)
#mod.bind(data_shapes=val.provide_data,
#        label_shapes=val.provide_label)    
#mod.set_params(arg_params, aux_params)
# mod.score(multi_val, Multi_Entropy(num=3),num_batch=100)
# mod.score(val, Multi_Entropy(num=3),num_batch=100)

# mod.score(val, AccuracyIlab,num_batch=1)

In [6]:
#mod._shared_module.bind(data_shapes=multi_val.provide_data,
#         label_shapes=multi_val.provide_label,force_rebind=True)   
#mod._shared_module.score(multi_val, Multi_Accuracy(num=3),num_batch=100)
#multi_val.provide_label

In [7]:
#mod._buckets[0]._symbol.list_arguments()
#mx.viz.plot_network(mod._shared_module._symbol, shape={"data":(1, 3, 224, 224)}, node_attrs={"shape":'rect',"fixedsize":'false'})

In [8]:
#mod._shared_module._symbol.list_arguments()

#multi_val.provide_label

In [9]:
#mod._shared_module._label_shapes = multi_val.provide_label
#mod._buckets[1]._label_shapes
#mod._buckets[0]._label_shapes

In [10]:
#model_prefix = '/home/ubuntu/results/ilab10_rand_multi'
#model_prefix = '/home/ubuntu/results/ilab_cat50'
#checkpoint = mx.callback.do_checkpoint(model_prefix)
#sym, arg_params, aux_params = \
#mx.model.load_checkpoint(model_prefix, 2)
 
#multi_val= Multi_ilab_iterator(val_raw,subset=[0,2,3])
#mod._shared_module.bind(data_shapes=multi_val.provide_data,\
#        label_shapes=multi_val.provide_label,force_rebind=True)  
#mod._shared_module.set_params(arg_params, aux_params)#

#mod._shared_module.score(multi_val, Multi_Accuracy(num=3),num_batch=100)
#multi_val.provide_label
#mod._shared_module._label_shapes.shape

In [11]:
a = mx.symbol.Variable('mona' ,init = mx.initializer.Xavier())

In [12]:
mx.symbol.Variable

<function mxnet.symbol.var>

In [13]:
#mod._buckets[0].keys_dict
#mod._buckets[0].idx2name
mod._buckets[0].score(val,Multi_Entropy(num=1), num_batch=100)
#mod._buckets[0]._arg_params

[<NDArray 64 @cpu(0)>]
1
(64,) (64, 10)
[<NDArray 64 @cpu(0)>]
1


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:110: RuntimeWarning: divide by zero encountered in log


(64,) (64, 10)
[<NDArray 64 @cpu(0)>]
1
(64,) (64, 10)
[<NDArray 64 @cpu(0)>]
1
(64,) (64, 10)


IndexError: index 10 is out of bounds for axis 1 with size 10

In [ ]:
#mod._buckets[2]._arg_params
diction0=mod._buckets[0].get_params()
diction1=mod._buckets[1].get_params()
#diction2=mod._buckets[2].get_params()
dictionGOD=mod._buckets[2].get_params()

In [ ]:
diction0[0]['fc1_bias'].asnumpy()

In [ ]:
diction1[0]['fc2_bias'].asnumpy()

In [ ]:
dictionGOD[0]['fc2_bias'].asnumpy()

In [ ]:
diction2[0]['fc3_bias'].asnumpy()

In [ ]:
mod._curr_module._param_names
aux_params = {name:[] for name in mod._curr_module._param_names}


In [ ]:
mod._buckets[2]._arg_params['bn0_beta'].asnumpy()

In [ ]:
mod._buckets[2]._param_names